# Ellipse Fitting

## 1) We extract the pixel coordinates of the droplet contour

We start by extracting the coordinates of the contour of the droplet.

To do this, we use one of the models we trained to generate a mask for a given image, then we extract the coordinates from that mask.

In [28]:
import os
import numpy as np
import cv2

# Directory with droplet masks (binary: 0 = background, 255 = droplet)
droplet_mask_dir = '../3. Segmentation and Detection Models/U-Net/droplet_unet'

# Pick a mask (example: first one)
mask_filename = sorted(os.listdir(droplet_mask_dir))[0]  # or loop over all
mask_path = os.path.join(droplet_mask_dir, mask_filename)

# Load binary droplet mask
mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

# Extract pixel coordinates of the droplet (where pixel == 255)
coords = np.column_stack(np.where(mask == 255))  # shape: (N, 2) → (y, x)

# Optional: convert (y, x) to (x, y) for geometric operations
coords = coords[:, [1, 0]]  # Now coords = (x, y)

# Print a few coordinates
print(f"Loaded mask: {mask_filename}, Droplet pixels: {len(coords)}")
print("Sample coordinates:", coords[:5])


Loaded mask: frame_000.png, Droplet pixels: 83754
Sample coordinates: [[360  87]
 [361  87]
 [362  87]
 [363  87]
 [364  87]]


## Compute covariance matrix

This gives us the spread and orientation of the droplet shape

In [ ]:
import numpy as np

# coords: (N, 2) array of (x, y) pixel positions

# Step 1: Center the coordinates (subtract the mean)
mean = np.mean(coords, axis=0)  # (x_mean, y_mean)
centered_coords = coords - mean  # each point minus the centroid

# Step 2: Compute the 2x2 covariance matrix
cov_matrix = np.cov(centered_coords.T)  # Transpose to shape (2, N)

print("Covariance Matrix:")
print(cov_matrix)


Covariance Matrix:
[[8857.92364424   22.1654505 ]
 [  22.1654505  5150.98254362]]


## Compute eigen values

In [30]:
 # Assuming `cov_matrix` is already computed
eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

# Sort eigenvalues and eigenvectors in descending order
order = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[order]
eigenvectors = eigenvectors[:, order]

# Ellipse parameters
major_axis_length = 2 * np.sqrt(eigenvalues[0])  # Scaling factor (optional)
minor_axis_length = 2 * np.sqrt(eigenvalues[1])
angle_rad = np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0])  # angle of major axis
angle_deg = np.degrees(angle_rad)

print("Major Axis Length:", major_axis_length)
print("Minor Axis Length:", minor_axis_length)
print("Orientation Angle (degrees):", angle_deg)


Major Axis Length: 188.23449393340982
Minor Axis Length: 143.538845073603
Orientation Angle (degrees): -179.657419367333


## Estimate contact angle

In [31]:
import numpy as np

# Use major/minor axis lengths from previous step
r = major_axis_length / 2  # base radius
h = minor_axis_length / 2  # droplet height

# Estimate radius of curvature
R = (r**2 + h**2) / (2 * h)

# Estimate contact angle in radians
theta_rad = np.arccos((R - h) / R)

# Convert to degrees
theta_deg = np.degrees(theta_rad)

print(f"Estimated Contact Angle: {theta_deg:.2f} degrees")


Estimated Contact Angle: 74.65 degrees
